# 2. PREPARACIÓN DE LOS DATOS

In [ ]:
# Importo librerias
import requests as r
import pandas as pd
import numpy as np
from dateutil.relativedelta import relativedelta

In [ ]:
df_items = pd.read_csv('../data/20220318_dataset_items.csv')

## CREACION DEL DATASET SELLERS
### Contiene los datos de los sellers únicos encontrados y el calculo de facturacion

In [ ]:
# se calcula el monto total por las ventas de cada item
df_items['sales_amount'] = df_items['price']*df_items['sold_quantity']

In [ ]:
# se calculo el 20% de ganancia sobre el monto total vendido de cada item
# en una segunda iteracion se propondrá mejorar el calculo de esta metrica
df_items['profits_amount'] = round(df_items['sales_amount']*0.20,1)

In [ ]:
metricas_sellers = df_items.groupby('seller',as_index=False).agg({'sales_amount': 'sum', 'sold_quantity':'sum', 'profits_amount': 'sum'})
metricas_sellers.rename(columns={
                'sales_amount': 'total_sales_amount',
                'sold_quantity': 'total_items_sold',
                'profits_amount': 'total_profits_amount'}, 
                        inplace=True)
metricas_sellers['avg_ticket'] = metricas_sellers['total_sales_amount']/metricas_sellers['total_items_sold']
metricas_sellers

In [ ]:
# se quitan duplicados para el subset de sellerID
sellers = df_items.drop_duplicates(subset='seller').reset_index(drop=True)
print("Cantidad de sellers: ", len(sellers))

In [ ]:
# Se eliminan las columnas que corresponden a items particulares
sellers.drop(columns={'price','sold_quantity', 'sales_amount', 'profits_amount'},inplace=True)

In [ ]:
# uno los datos calculados de facturacion y total de items vendidos
sellers = sellers.merge(metricas_sellers)

In [ ]:
# le uno al dataset final la columna con el nombre de la categoria
# para hacer mas comprensible la proxima etapa
sellers = sellers.merge(categorias)

In [ ]:
# transformo la columna de fecha de registro a date
sellers['register_date'] = pd.to_datetime(sellers['fecha_registro']).dt.date

In [ ]:
# calculo la antiguedad en años
def ages(register):
    antiguedad = relativedelta(pd.to_datetime('now'), register) 
    return '{}'.format(antiguedad.years)

sellers['antiquity_ages'] = sellers["register_date"].apply(ages)

In [ ]:
# calculamos el ratio de cancelaciones sobre el total vendido
sellers['transactions_canceled_historic_rate'] = sellers['transactions_canceled_historic']/sellers['transactions_total_historic']

In [ ]:
# extraigo el dataset
sellers.to_csv('../data/20220318_dataset_sellers.csv',index=False)